## Creating Context for the LLM

In [44]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import os 
from dotenv import load_dotenv


In [45]:
load_dotenv()
groq_api_key=os.getenv('GROQ_API_KEY')

In [46]:
loader = PyPDFLoader("Ip102_Impact.pdf")
pages = loader.load()

In [47]:
rows = []
for page in pages:
    for line in page.page_content.split("\n"):
        parts = line.strip().split()
        if len(parts) >= 5 and parts[0].lower() != "pest":
            rows.append(line.strip())

In [50]:
docs = [Document(page_content=row) for row in rows]
print("Sample document:", docs[32])

Sample document: page_content='Alfalfa Caterpillar Alfalfa High 18 12'


In [54]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [55]:
vector_db = FAISS.from_documents(docs,embeddings)
vector_db
vector_db.save_local('faiss_index')

KeyboardInterrupt: 

In [56]:
rst=vector_db.similarity_search('Corn Leaf Miner Corn Medium')
rst[1].page_content

'Corn Leaf Miner Corn Medium 14 10'

In [57]:
retriever= vector_db.as_retriever()

In [58]:
retriever.invoke("mango")

[Document(id='5329a0d5-bd7f-4058-bd5e-96bc651037dd', metadata={}, page_content='Mango Leaf Webber Mango Medium 14 10'),
 Document(id='e1fd87ad-9f93-4783-b4f0-0f68a3160848', metadata={}, page_content='Mango Hopper Mango Medium 12 8'),
 Document(id='e0819c19-687f-408c-bb0e-d36439e33ce5', metadata={}, page_content='Mango Twig Borer Mango Medium 12 8'),
 Document(id='2643b2e0-e1cd-4fef-89e6-a13b9483f63d', metadata={}, page_content='Mango Hopper Mango High 22 18')]

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_groq import ChatGroq
from langchain_core.documents import Document

In [ ]:
llm = ChatGroq(api_key=groq_api_key, model='Gemma2-9b-It')

In [ ]:
prompt= ChatPromptTemplate.from_messages([
    ("system","You are an impact analyzer agent for a smart farming system," 
     "use the context below to provide accurate and precise impact prediction based on the user input."
     "provide the user with Risk Level, Avg Damage, Predicted Yield Loss."),
     ("system","<context>\n{context}\n</context>"),
     ("human", "Question:{input} and {croptype}")
])

In [ ]:
chain = create_stuff_documents_chain(llm,prompt)

In [ ]:
chain.invoke({
      "input": "Green Rice Leafhopper ",
      "croptype":"rice",
      "context":[Document(page_content="'Green Rice LeafhopperRice Medium 12 8\nRice Gall Midge Rice High 18 12\nRice Stem Borer Rice High 22 17\nRice Brown LeafhopperRice Medium 14 10\nRice Green LeafhopperRice Medium 12 8')")]

})

"Based on the provided context, here's the impact prediction for Green Rice Leafhopper on rice:\n\n**Pest:** Green Rice Leafhopper\n\n**Crop:** Rice\n\n**Risk Level:** Medium\n\n**Avg Damage:** 8 \n\n**Predicted Yield Loss:** 12%\n\n\n**Explanation:**\n\nThe context shows that Green Rice Leafhopper has a Medium risk level and causes an average damage of 8 on a rice crop, leading to a predicted yield loss of 12%. \n\n**Important Note:**\n\nThis prediction is based solely on the limited data provided. \n\nFor more accurate and precise impact analysis, consider these factors:\n\n* **Specific rice variety:** Different rice varieties have varying levels of susceptibility to pests.\n* **Geographic location:** Pest pressure and environmental conditions can vary significantly by location.\n* **Growth stage of the rice crop:** Pests can have different impacts at different growth stages.\n* **Pest population density:** The number of pests present directly influences the potential damage.\n* **Co

In [ ]:
ret_chain = create_retrieval_chain(retriever,chain)
ret_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026122C92140>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'croptype', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an impact analyzer agent for a smart farming system,use the context below to provide accurate and precise impact prediction base

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
result = ret_chain.invoke({    "input": "Green Rice Leafhopper", "croptype": "Rice" })
result['answer']


"Based on the provided context, here's the impact prediction for Green Rice Leafhopper on Rice:\n\n**Pest Name:** Green Rice Leafhopper\n**Crop Type:** Rice\n**Risk Level:** Medium\n**Avg Damage %:** 8\n**Predicted Yield Loss %:** 12 \n\n**Explanation:**\n\nThe context data shows that Green Rice Leafhopper  affects rice with a medium risk level, causing an average damage of 8% and a predicted yield loss of 12%. \n\n\nLet me know if you have any other pests or crops you'd like to analyze! \n"

In [ ]:
parser =StrOutputParser()

In [ ]:
from langchain_core.runnables import RunnableLambda
chain = ret_chain | RunnableLambda(lambda x: x["answer"]) | parser

In [ ]:
parsed = chain.invoke({"input": "Green Rice Leafhopper","croptype":" Rice"})
print(parsed)

## Impact Prediction for Green Rice Leafhopper on Rice:

Based on the provided context, here's the impact prediction for Green Rice Leafhopper on Rice:

* **Pest Name:** Green Rice Leafhopper
* **Crop Type:** Rice
* **Risk Level:** Medium
* **Avg Damage %:** 8
* **Predicted Yield Loss %:** 8 


Let me know if you have any other pests or crops you'd like to analyze! 



## Save the models and artifacts